# StockDatabaseのデバッグ 

## 実行位置の調整

ルートディレクトリで行う

In [1]:
%cd ..

E:\システムトレード入門\trade_system_git_workspace


## インポート

In [2]:
from get_stock_price import YahooFinanceStockLoaderMin

In [3]:
from get_stock_price.stock_db_ver2 import StockDatabase

In [4]:
from pathlib import Path
import datetime
from pytz import timezone
from contextlib import closing

In [5]:
from line_profiler import LineProfiler

In [6]:
from utils import ConvertFreqOHLCV

In [7]:
from utils import get_previous_datetime, get_next_datetime

## データの取得 

In [15]:
stock_names  = ["4755.T","6502.T","2802.T","6954.T"]
#stock_names = "6502.T"


stockloader = YahooFinanceStockLoaderMin(stock_names, stop_time_span=2.0, is_use_stop=False, to_tokyo=True)
stock_df = stockloader.load()
stock_df.tail(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_2802,High_2802,Low_2802,Close_2802,Volume_2802,Open_6954,High_6954,Low_6954,Close_6954,Volume_6954
timestamp,,,,,,,,,,,,,,,,,,,,
2021-03-25 14:55:00+09:00,1343.0,1343.0,1342.0,1342.0,17700.0,3825.0,3825.0,3820.0,3825.0,4800.0,2332.0,2334.5,2332.0,2332.5,4100.0,26205.0,26225.0,26200.0,26210.0,5000.0
2021-03-25 14:56:00+09:00,1342.0,1343.0,1341.0,1342.0,74500.0,3825.0,3825.0,3820.0,3825.0,5900.0,2333.0,2333.5,2332.0,2333.0,3500.0,26200.0,26220.0,26200.0,26220.0,2700.0
2021-03-25 14:57:00+09:00,1343.0,1343.0,1343.0,1343.0,6700.0,3825.0,3825.0,3825.0,3825.0,8700.0,2333.0,2333.0,2330.5,2332.5,11000.0,26215.0,26215.0,26180.0,26185.0,6900.0
2021-03-25 14:58:00+09:00,1343.0,1343.0,1342.0,1342.0,24500.0,3825.0,3825.0,3820.0,3825.0,30000.0,2331.5,2332.5,2330.5,2331.5,5700.0,26190.0,26215.0,26190.0,26205.0,5900.0
2021-03-25 14:59:00+09:00,1343.0,1343.0,1337.0,1337.0,252600.0,3825.0,3830.0,3815.0,3820.0,35200.0,2331.0,2334.5,2330.5,2334.5,15500.0,26205.0,26220.0,26155.0,26165.0,20300.0


In [16]:
stock_df.head(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502,Open_2802,High_2802,Low_2802,Close_2802,Volume_2802,Open_6954,High_6954,Low_6954,Close_6954,Volume_6954
timestamp,,,,,,,,,,,,,,,,,,,,
2021-03-22 09:00:00+09:00,1468.0,1479.0,1464.0,1472.0,0.0,3950.0,3965.0,3935.0,3960.0,0.0,2316.0,2322.0,2313.0,2321.0,0.0,26310.0,26410.0,26235.0,26235.0,0.0
2021-03-22 09:01:00+09:00,1471.0,1479.0,1471.0,1472.0,91900.0,3960.0,3960.0,3950.0,3950.0,25100.0,2322.0,2324.5,2318.5,2321.0,8000.0,26255.0,26275.0,26110.0,26135.0,11300.0
2021-03-22 09:02:00+09:00,1473.0,1479.0,1471.0,1477.0,85000.0,3950.0,3975.0,3950.0,3965.0,43200.0,2322.0,2325.0,2322.0,2324.5,3900.0,26110.0,26205.0,26045.0,26165.0,15800.0
2021-03-22 09:03:00+09:00,1478.0,1479.0,1472.0,1474.0,69900.0,3970.0,3980.0,3950.0,3970.0,59500.0,2325.0,2332.0,2325.0,2330.0,24000.0,26160.0,26175.0,26105.0,26125.0,5300.0
2021-03-22 09:04:00+09:00,1473.0,1478.0,1472.0,1475.0,39800.0,3970.0,3980.0,3970.0,3975.0,46500.0,2331.0,2332.0,2325.5,2328.5,6100.0,26105.0,26105.0,26015.0,26085.0,10400.0


## データベースオブジェクト

In [17]:
db_path = Path("db/sample_db/sample.db")
stock_db = StockDatabase(db_path, database_frequency="T")

In [18]:
stock_db.database_frequency

'T'

###  データのインサート

In [19]:
stock_db.upsert(stock_df, item_replace_type="replace_null")

In [20]:
stock_db.sammary_column_arrange()

,column_name,arrange_table
0,Close_2802,table_0
1,Close_4755,table_0
2,Close_6502,table_0
3,Close_6954,table_0
4,High_2802,table_0
5,High_4755,table_0
6,High_6502,table_0
7,High_6954,table_0
8,Low_2802,table_0
9,Low_4755,table_0


In [21]:
stock_db.sammary_timestamp(to_tokyo=False)

,table_name,min_datetime,max_datetime
0,table_0,2021-03-22 00:00:00+00:00,2021-03-25 05:59:00+00:00


utcで表示されていることに注意

In [22]:
stock_db.database_frequency

'T'

### search_span 

In [10]:
#stock_names = "6502"
stock_names = ["4755","6502"]

#start_datetime = datetime.datetime(2020, 10, 30, 0, 0, 0)
#end_datetime = datetime.datetime(2020, 11, 2, 0, 0, 0)

utc_timezone = timezone("UTC")
start_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))
end_datetime = utc_timezone.localize(datetime.datetime(2020, 11, 2, 0, 0, 0))

selected_df = stock_db.search_span(stock_names=stock_names,
                                   start_datetime=start_datetime,
                                   end_datetime=end_datetime,
                                   freq_str="10T",
                                   to_tokyo=False
                                  )
selected_df.head(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,,,,,,
2020-10-30 00:00:00+00:00,1050.0,1063.0,1043.0,1053.0,560700.0,2695.0,2696.0,2668.0,2675.0,41100.0
2020-10-30 00:10:00+00:00,1053.0,1054.0,1047.0,1047.0,306200.0,2677.0,2686.0,2672.0,2672.0,27800.0
2020-10-30 00:20:00+00:00,1047.0,1051.0,1044.0,1049.0,246400.0,2674.0,2677.0,2666.0,2667.0,36600.0
2020-10-30 00:30:00+00:00,1049.0,1049.0,1039.0,1041.0,484700.0,2666.0,2669.0,2660.0,2666.0,37900.0
2020-10-30 00:40:00+00:00,1040.0,1040.0,1035.0,1037.0,208300.0,2665.0,2666.0,2653.0,2653.0,25600.0


viewを用いた場合

In [17]:
#with stock_db.create_view(stock_names=stock_names, start_datetime=start_datetime, end_datetime=end_datetime) as view:
with stock_db.create_view(stock_names=stock_names, start_datetime=None, end_datetime=None) as view:
    select_df = stock_db.search_span(stock_names=stock_names,
                                     view=view,
                                     start_datetime=start_datetime,
                                     end_datetime=end_datetime,
                                     freq_str="10T",
                                     to_tokyo=False,
                                     )
    print(view.view_name)
select_df.head(5)

view_4755_6502


,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,,,,,,
2020-10-30 00:00:00+00:00,1050.0,1063.0,1043.0,1053.0,560700.0,2695.0,2696.0,2668.0,2675.0,41100.0
2020-10-30 00:10:00+00:00,1053.0,1054.0,1047.0,1047.0,306200.0,2677.0,2686.0,2672.0,2672.0,27800.0
2020-10-30 00:20:00+00:00,1047.0,1051.0,1044.0,1049.0,246400.0,2674.0,2677.0,2666.0,2667.0,36600.0
2020-10-30 00:30:00+00:00,1049.0,1049.0,1039.0,1041.0,484700.0,2666.0,2669.0,2660.0,2666.0,37900.0
2020-10-30 00:40:00+00:00,1040.0,1040.0,1035.0,1037.0,208300.0,2665.0,2666.0,2653.0,2653.0,25600.0


### search_one 

In [18]:
#stock_names = "4755"
stock_names = "6502"
select_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))

selected_df = stock_db.search_one(
                                  stock_names=stock_names,
                                  select_datetime=select_datetime,
                                  freq_str="10T",
                                  to_tokyo=False
                                  )
selected_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-10-30 00:00:00+00:00,2695.0,2696.0,2668.0,2675.0,41100.0


viewを用いた場合

In [19]:
with stock_db.create_view(stock_names=stock_names, start_datetime=None, end_datetime=None) as view:
    selected_df = stock_db.search_one(
                                      stock_names=stock_names,
                                      view=view,
                                      select_datetime=select_datetime,
                                      freq_str="10T",
                                      to_tokyo=False
                                      )
selected_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-10-30 00:00:00+00:00,2695.0,2696.0,2668.0,2675.0,41100.0


### search_iter 

In [20]:
#stock_names = "4755"
stock_names = "6502"
from_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))

df_gen = stock_db.search_iter(
                              stock_names=stock_names,
                              from_datetime=from_datetime,
                              freq_str="10T"
                             )


In [22]:
next(df_gen)

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-10-30 09:00:00+09:00,2695.0,2696.0,2668.0,2675.0,41100.0


viewを用いた場合

In [23]:
with stock_db.create_view(stock_names=stock_names, start_datetime=None, end_datetime=None) as view:
    df_gen = stock_db.search_iter(
                                  stock_names=stock_names,
                                  view=view,
                                  from_datetime=from_datetime,
                                  freq_str="10T"
                                 )
    selected_df = next(df_gen)
selected_df

,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,
2020-10-30 09:00:00+09:00,2695.0,2696.0,2668.0,2675.0,41100.0


###   StockDatabase.get_all_content_gen

In [24]:
get_all_content_gen = StockDatabase.get_all_content_gen(db_path, has_arrange_count=True, column_max=1000, row_max=1000)

In [25]:
next(get_all_content_gen)

,Close_2802,Close_4755,Close_6502,Close_6954,High_2802,High_4755,High_6502,High_6954,Low_2802,Low_4755,Low_6502,Low_6954,Open_2802,Open_4755,Open_6502,Open_6954,Volume_2802,Volume_4755,Volume_6502,Volume_6954
timestamp,,,,,,,,,,,,,,,,,,,,
2020-10-30 00:00:00+00:00,2107.5,1048.0,2686.0,NaN,2117.0,1050.0,2696.0,NaN,2107.0,1043.0,2686.0,NaN,2117.0,1050.0,2695.0,NaN,0.0,0.0,0.0,NaN
2020-10-30 00:01:00+00:00,2108.0,1054.0,2684.0,NaN,2109.5,1055.0,2691.0,NaN,2105.5,1047.0,2683.0,NaN,2106.0,1048.0,2691.0,NaN,7000.0,66300.0,7000.0,NaN
2020-10-30 00:02:00+00:00,2111.5,1056.0,2683.0,NaN,2112.5,1056.0,2687.0,NaN,2108.0,1048.0,2681.0,NaN,2108.0,1053.0,2683.0,NaN,13200.0,200000.0,3800.0,NaN
2020-10-30 00:03:00+00:00,2116.5,1055.0,2683.0,NaN,2116.5,1063.0,2687.0,NaN,2111.5,1055.0,2683.0,NaN,2111.5,1057.0,2686.0,NaN,10200.0,141800.0,1600.0,NaN
2020-10-30 00:04:00+00:00,2117.0,1054.0,2676.0,NaN,2117.0,1058.0,2683.0,NaN,2115.0,1053.0,2675.0,NaN,2115.5,1056.0,2683.0,NaN,2500.0,27300.0,8200.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-30 16:35:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-30 16:36:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-10-30 16:37:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### StockDatabase.load_db 

In [ ]:
db_path2 = Path("db/sample_db") / Path("sample2.db")  # 空のデータベース
db_path_for_load = Path("db/for_load_db") / Path("stock.db")  # そこそこ大きなデータベース

stock_db2 = StockDatabase(db_path2, database_frequency="T")

In [ ]:
stock_db2.load_other_db(db_path_for_load, has_arrange_count=True, column_max=500, row_max=500, item_replace_type="replace_null")

In [ ]:
stock_db2.sammary_column_arrange()

### search_span_all_iter 

In [14]:
#stock_names = "6502"
stock_names = ["4755","6502"]

#start_datetime = datetime.datetime(2020, 10, 30, 0, 0, 0)
#end_datetime = datetime.datetime(2020, 11, 2, 0, 0, 0)

utc_timezone = timezone("UTC")
start_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))
end_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 2, 0, 0))

span_all_gen = stock_db.search_span_all_iter(stock_names=stock_names,
                                   start_datetime=start_datetime,
                                   end_datetime=end_datetime,
                                   row_max=10,
                                   column_max=1,
                                   to_tokyo=True,
                                   )

In [15]:
for stock_df in span_all_gen:
    print(stock_df)

                           Open_4755  High_4755  Low_4755  Close_4755  \
timestamp                                                               
2020-10-30 09:00:00+09:00     1050.0     1050.0    1043.0      1048.0   
2020-10-30 09:01:00+09:00     1048.0     1055.0    1047.0      1054.0   
2020-10-30 09:02:00+09:00     1053.0     1056.0    1048.0      1056.0   
2020-10-30 09:03:00+09:00     1057.0     1063.0    1055.0      1055.0   
2020-10-30 09:04:00+09:00     1056.0     1058.0    1053.0      1054.0   
2020-10-30 09:05:00+09:00     1054.0     1057.0    1053.0      1055.0   
2020-10-30 09:06:00+09:00     1055.0     1056.0    1053.0      1054.0   
2020-10-30 09:07:00+09:00     1055.0     1055.0    1054.0      1054.0   
2020-10-30 09:08:00+09:00     1054.0     1055.0    1053.0      1053.0   
2020-10-30 09:09:00+09:00     1054.0     1054.0    1052.0      1053.0   

                           Volume_4755  
timestamp                               
2020-10-30 09:00:00+09:00          0.0  


2020-10-30 10:29:00+09:00      18100.0  
                           Open_4755  High_4755  Low_4755  Close_4755  \
timestamp                                                               
2020-10-30 10:30:00+09:00     1030.0     1033.0    1030.0      1033.0   
2020-10-30 10:31:00+09:00     1032.0     1033.0    1031.0      1032.0   
2020-10-30 10:32:00+09:00     1032.0     1033.0    1032.0      1033.0   
2020-10-30 10:33:00+09:00     1033.0     1034.0    1031.0      1032.0   
2020-10-30 10:34:00+09:00     1031.0     1033.0    1031.0      1032.0   
2020-10-30 10:35:00+09:00     1033.0     1033.0    1030.0      1031.0   
2020-10-30 10:36:00+09:00     1031.0     1031.0    1030.0      1030.0   
2020-10-30 10:37:00+09:00     1031.0     1032.0    1030.0      1030.0   
2020-10-30 10:38:00+09:00     1031.0     1031.0    1031.0      1031.0   
2020-10-30 10:39:00+09:00     1031.0     1031.0    1030.0      1031.0   

                           Volume_4755  
timestamp                               


2020-10-30 10:09:00+09:00        100.0  
                           Open_6502  High_6502  Low_6502  Close_6502  \
timestamp                                                               
2020-10-30 10:10:00+09:00     2650.0     2650.0    2650.0      2650.0   
2020-10-30 10:11:00+09:00     2649.0     2649.0    2649.0      2649.0   
2020-10-30 10:12:00+09:00     2649.0     2651.0    2648.0      2651.0   
2020-10-30 10:13:00+09:00     2651.0     2652.0    2650.0      2650.0   
2020-10-30 10:14:00+09:00     2650.0     2650.0    2649.0      2650.0   
2020-10-30 10:15:00+09:00     2649.0     2653.0    2649.0      2650.0   
2020-10-30 10:16:00+09:00     2650.0     2651.0    2650.0      2650.0   
2020-10-30 10:17:00+09:00     2650.0     2650.0    2649.0      2649.0   
2020-10-30 10:18:00+09:00     2650.0     2650.0    2649.0      2649.0   
2020-10-30 10:19:00+09:00     2650.0     2650.0    2649.0      2649.0   

                           Volume_6502  
timestamp                               


### make_db 

In [10]:
#stock_names = "6502"
stock_names = ["4755","6502"]

#start_datetime = datetime.datetime(2020, 10, 30, 0, 0, 0)
#end_datetime = datetime.datetime(2020, 11, 2, 0, 0, 0)

utc_timezone = timezone("UTC")
start_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))
end_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 2, 0, 0))

new_db_path = Path("db/sample_db") / Path("sample3.db")

new_stock_db = stock_db.make_db(new_db_path,
                                stock_names=stock_names,
                                start_datetime=start_datetime,
                                end_datetime=end_datetime,
                                row_max=100,
                                column_max=2,
                                to_tokyo=False
                               )

count :1

In [11]:
selected_df = new_stock_db.search_span(stock_names=stock_names,
                         start_datetime=start_datetime,
                         end_datetime=end_datetime,
                         freq_str="10T",
                         to_tokyo=False
                        )
selected_df.head(5)

,Open_4755,High_4755,Low_4755,Close_4755,Volume_4755,Open_6502,High_6502,Low_6502,Close_6502,Volume_6502
timestamp,,,,,,,,,,
2020-10-30 00:00:00+00:00,1050.0,1063.0,1043.0,1053.0,560700.0,2695.0,2696.0,2668.0,2675.0,41100.0
2020-10-30 00:10:00+00:00,1053.0,1054.0,1047.0,1047.0,306200.0,2677.0,2686.0,2672.0,2672.0,27800.0
2020-10-30 00:20:00+00:00,1047.0,1051.0,1044.0,1049.0,246400.0,2674.0,2677.0,2666.0,2667.0,36600.0
2020-10-30 00:30:00+00:00,1049.0,1049.0,1039.0,1041.0,484700.0,2666.0,2669.0,2660.0,2666.0,37900.0
2020-10-30 00:40:00+00:00,1040.0,1040.0,1035.0,1037.0,208300.0,2665.0,2666.0,2653.0,2653.0,25600.0


## 小規模データベースのプロファイリング

### upsert のプロファイリング

In [ ]:
def upsert():
    stock_db.upsert(stock_df, item_replace_type="replace_null")

In [ ]:
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)                                                                                          
prf.add_function(upsert)                                                                                      
prf.runcall(upsert)                                                                                            
prf.print_stats()

### searchのプロファイリング 

In [ ]:
#stock_names = ["4755","6502"]
stock_names = ["4755","6502"]

start_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))
end_datetime = utc_timezone.localize(datetime.datetime(2020, 11, 2, 0, 0, 0))

def search():
    for i in range(10):
        selected_df = stock_db.search_span(stock_names=stock_names,
                                           start_datetime=start_datetime,
                                           end_datetime=end_datetime,
                                           freq_str="10T",
                                           to_tokyo=False
                                          )

In [ ]:
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)
#prf.add_module(ConvertFreqOHLCV)
prf.add_function(search)                                                                                      
prf.runcall(search)                                                                                            
prf.print_stats()

## 大規模データベースのプロファイリング 

In [ ]:
#big_sample_db = Path("db/big_sample_db/") / Path("stock.db")
big_sample_db = Path("db/stock_db/") / Path("stock.db")
big_db = StockDatabase(big_sample_db)

In [ ]:
big_db.sammary_column_arrange()

### upsertのプロファイリング 

In [ ]:
def upsert_big():
    big_db.upsert(stock_df, item_replace_type="replace_null")

In [ ]:
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)                                                                                          
prf.add_function(upsert_big)                                                                                      
prf.runcall(upsert_big)                                                                                            
prf.print_stats()

### search_spanのプロファイリング 

In [ ]:
stock_names = ["2531","1679","2607","3470","4284","5199","6308","7088","7952","9048"]
utc_timezone = timezone("UTC")
start_datetime = utc_timezone.localize(datetime.datetime(2020,11,17,0,0,0))
end_datetime = utc_timezone.localize(datetime.datetime(2020,11,18,0,0,0))

def search_big():
    for i in range(10):
        selected_df = big_db.search_span(stock_names=stock_names,
                                         start_datetime=start_datetime,
                                         end_datetime=end_datetime,
                                         freq_str="10T",
                                         to_tokyo=False
                                        )

In [ ]:
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)
#prf.add_module(ConvertFreqOHLCV)
prf.add_function(search_big)                                                                                      
prf.runcall(search_big)                                                                                            
prf.print_stats()

### search_oneのプロファイリング 

In [ ]:
stock_names = "9997"
#select_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))
select_datetime = datetime.datetime(2020, 10, 30, 0, 0, 0)


def search_one():
    for i in range(10):
        selected_df = big_db.search_one(
                                        stock_names=stock_names,
                                        select_datetime=select_datetime,
                                        freq_str="10T",
                                        to_tokyo=False
                                        )
    
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)
#prf.add_module(ConvertFreqOHLCV)
prf.add_function(search_one)                                                                                      
prf.runcall(search_one)                                                                                  
prf.print_stats()

### search_iterのプロファイリング 

In [ ]:
stock_names = "9997"
from_datetime = utc_timezone.localize(datetime.datetime(2020, 10, 30, 0, 0, 0))

df_gen = big_db.search_iter(
                            stock_names=stock_names,
                            from_datetime=from_datetime,
                            freq_str="1H"
                            )
next(df_gen)

In [ ]:
def next_iter():
    next(df_gen)
    
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)
#prf.add_module(ConvertFreqOHLCV)
prf.add_function(next_iter)                                                                                      
prf.runcall(next_iter)                                                                                  
prf.print_stats()

### search_span(viewあり)のプロファイリング 

In [ ]:
stock_names = ["2531","1679","2607","3470","4284","5199","6308","7088","7952","9048"]
utc_timezone = timezone("UTC")
start_datetime = utc_timezone.localize(datetime.datetime(2020,11,17,0,0,0))
end_datetime = utc_timezone.localize(datetime.datetime(2020,11,18,0,0,0))

def search_big():
    with big_db.create_view(stock_names, start_datetime=start_datetime, end_datetime=end_datetime) as view:
        for i in range(10):
            selected_df = big_db.search_span(stock_names=stock_names,
                                             start_datetime=start_datetime,
                                             end_datetime=end_datetime,
                                             freq_str="10T",
                                             to_tokyo=False,
                                             view=view
                                            )

In [ ]:
prf = LineProfiler()                                                                                         
prf.add_module(StockDatabase)
#prf.add_module(ConvertFreqOHLCV)
prf.add_function(search_big)                                                                                      
prf.runcall(search_big)                                                                                            
prf.print_stats()

##  データベースのサマリー

In [ ]:
stock_array = stock_db.sammary_stocklist()
stock_array

In [17]:
stock_names = ["4755","6502"]
stock_timestamp = stock_db.stock_timestamp(stock_names=stock_names)
stock_timestamp

,min_datetime,max_datetime,column_name
0,2020-10-30 00:00:00+00:00,2020-12-29 05:59:00+00:00,4755
1,2020-10-30 00:00:00+00:00,2020-12-29 05:59:00+00:00,6502


In [ ]:
stock_timestamp = big_db.stock_timestamp(stock_names)
stock_timestamp